In [0]:
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout
from keras.layers import Flatten, GlobalAveragePooling2D
from keras.preprocessing import image
from keras import regularizers
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os
import pandas as pd

In [0]:
# example of progressively loading images from file
from keras.preprocessing.image import ImageDataGenerator

dir_path = 'train_set/'
# numbers are based on mode, varified with histogram
img_width, img_height = 224, 224

# create generator
datagen = ImageDataGenerator(validation_split=0.1)
# prepare an iterators for each dataset
train_it = datagen.flow_from_directory(dir_path, 
                                       batch_size=64,
                                       target_size=(img_width, img_height),
                                       subset='training')
val_it = datagen.flow_from_directory(dir_path,
                                     batch_size=64,
                                     target_size=(img_width, img_height),
                                     subset='validation')
# confirm the iterator works
batchX, batchy = train_it.next()
val_batchX, val_batchy  = val_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

In [0]:
# model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
model_xception = Xception(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

In [0]:
# add a global spatial average pooling layer
x = model_xception.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)
x = Dropout(0.3)(x)

# Make an output layer for our 4 classes.
predictions = Dense(4, activation='softmax', kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01))(x)

In [0]:
model = Model(inputs=model_xception.inputs, outputs=predictions)

In [0]:
# Mark some layers as not trainable. We experimented with freezing some layers from the pretrained models in order to retain the learned knowledge as it is.

# model.get_layer('block1_conv1').trainable = False
# model.get_layer('block1_conv2').trainable = False
# model.get_layer('block2_sepconv1').trainable = False
# model.get_layer('block2_sepconv2').trainable = False
# model.get_layer('conv2d_1').trainable = False

for layer in model_xception.layers:
    layer.trainable = False


In [0]:
model.summary()

In [0]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log_xception.csv', append=True, separator=';')

In [0]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
model.fit_generator(
    train_it,
    epochs=25,
    validation_data=val_it,
    callbacks=[csv_logger]
)

In [0]:
# model.save('xception_model_tf.h5')

In [0]:
model.summary()

In [0]:
def create_test_images_array():
    
    images = []
    
    for test_i in os.listdir("test_set/"):
        img_path = "test_set/"+test_i
        img = image.load_img(img_path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        images.append(x)
    return images

In [0]:
test_images = create_test_images_array()

In [0]:
model.predict(preprocess_input(np.expand_dims(image.img_to_array(image.load_img("test_set/test_1.jpg", target_size=(500,400))), axis=0)))

In [0]:
predictions = [model.predict(im) for im in test_images]

In [0]:
prediction_classes = [np.argmax(ar) for ar in predictions]

In [0]:
prediction_classes

In [0]:
mapping = {0: "funny", 1: "hilarious", 2: "not_funny", 3: "very_funny"}

In [0]:
predictions_final = [mapping[cl] for cl in prediction_classes]

In [0]:
predictions_final

In [0]:
real_test_set = pd.read_csv("test_data.csv")

In [0]:
real_test_set

In [0]:
real_test_set["Humour"] = predictions_final

In [0]:
def make_predictions_df(test_data_with_pred):
    df = test_data_with_pred[["id", "Humour"]]
    return df

In [0]:
make_predictions_df(real_test_set).to_csv("submission_19_xception.csv", index=False)